In [48]:
import pandas as pd
import numpy as np
from __future__ import print_function
from pyspark.ml.classification import GBTClassifier
import findspark
findspark.init()
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession
from pyspark.sql import Row
import collections
from pyspark import SparkConf,SparkContext

df1=pd.read_csv('churn.csv')

df1

df1.drop(['Names','Unnamed: 0'],axis=1,inplace=True)

df1['Age']=df1['Age'].astype(float)
df1['Total_Purchase']=df1['Total_Purchase'].astype(float)
df1['Account_Manager']=df1['Account_Manager'].astype(float)
df1['Years']=df1['Years'].astype(float)
df1['Num_Sites']=df1['Num_Sites'].astype(float)
df1['Churn']=df1['Churn'].astype(float)

df1.info()

df1.to_csv('churnprep.csv')

df1

In [49]:
spark=SparkSession.builder.getOrCreate()

In [50]:
inputlines=df = spark.read.options(header = True, inferSchema = True).csv("churnprep.csv") #spark.sparkContext.textFile('churnprep1.csv')

In [51]:
inputlines

DataFrame[index: int, Age: double, Total_Purchase: double, Account_Manager: double, Years: double, Num_Sites: double, Churn: double]

In [52]:
#data=inputlines.map(lambda x: x.split(","))

In [53]:
#data

In [54]:
#colnames=['Age','Total_Purchase','Account_Manager','Years','Num_Sites','Churn']

In [55]:
#df=inputlines.toDF(colnames)

In [56]:
df = inputlines
df

DataFrame[index: int, Age: double, Total_Purchase: double, Account_Manager: double, Years: double, Num_Sites: double, Churn: double]

In [57]:
from  pyspark.ml.feature import VectorAssembler

In [58]:
vecAssembler = VectorAssembler(inputCols=['Age','Total_Purchase','Account_Manager','Years','Num_Sites'], outputCol="features")
df = vecAssembler.transform(df)
df

DataFrame[index: int, Age: double, Total_Purchase: double, Account_Manager: double, Years: double, Num_Sites: double, Churn: double, features: vector]

In [59]:
df = df.drop("index", 'Age','Total_Purchase','Account_Manager','Years','Num_Sites')
df

DataFrame[Churn: double, features: vector]

In [60]:
df = df.withColumnRenamed("Churn", "label")
df

DataFrame[label: double, features: vector]

In [61]:
trainTest=df.randomSplit([0.5,0.5])
trainingDF=trainTest[0]
testDF=trainTest[1]

In [62]:
trainingDF

DataFrame[label: double, features: vector]

In [63]:
testDF

DataFrame[label: double, features: vector]

In [64]:

from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

In [65]:
gbt=GBTClassifier(featuresCol='features',maxIter=10)

In [66]:
gbt = gbt.fit(trainingDF)

In [67]:
predictions = gbt.transform(testDF)
predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|[22.0,11254.38,1....|[1.02123000176285...|[0.88518352410328...|       0.0|
|  0.0|[25.0,9672.03,0.0...|[0.92554866753198...|[0.86425589122228...|       0.0|
|  0.0|[27.0,8628.8,1.0,...|[0.85489526677014...|[0.84680911349936...|       0.0|
|  0.0|[28.0,8670.98,0.0...|[1.13424217255855...|[0.90623307473440...|       0.0|
|  0.0|[29.0,9378.24,0.0...|[0.98154842037103...|[0.87686771101437...|       0.0|
|  0.0|[29.0,10203.18,1....|[0.91343241063173...|[0.86138781405375...|       0.0|
|  0.0|[29.0,13240.01,1....|[1.20926313553665...|[0.91822915870950...|       0.0|
|  0.0|[29.0,13255.05,1....|[1.02017899862339...|[0.88496971649468...|       0.0|
|  0.0|[30.0,6744.87,0.0...|[1.24577770983735...|[0.92354769908127...|       0.0|
|  0.0|[30.0,867

In [68]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)


Accuracy: 0.8657074340527577
